In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv(r"/content/drive/MyDrive/BDI LAB/Paraphrased/NSF_FAQ_Processed_Question.csv")

In [ ]:
data= data.drop(data.columns[0],axis=1)
data.head()

,Topics,Questions
0,PROGRAM BASICS,What are the SBIR and STTR Programs? What are ...
1,PROGRAM BASICS,What are the funding priorities for NSF SBIR/S...
2,PROGRAM BASICS,What is innovative research and how do I gauge...
3,PROGRAM BASICS,How does NSF select which projects to support?...
4,PROGRAM BASICS,What activities and expenses are appropriate t...


In [ ]:
# !pip install transformers


In [ ]:
# !pip install pytorch_lightning==0.7.5

In [ ]:
# from transformers import AutoModelWithLMHead, AutoTokenizer

In [ ]:
# tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-small-finetuned-quora-for-paraphrasing")
# model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-small-finetuned-quora-for-paraphrasing")

In [ ]:
def paraphrase(text, max_length=128):
  input_ids = tokenizer.encode(text, return_tensors="pt", add_special_tokens=True)
  generated_ids = model.generate(input_ids=input_ids, num_return_sequences=5, num_beams=5, max_length=max_length, no_repeat_ngram_size=2, repetition_penalty=3.5, length_penalty=1.0, early_stopping=True)

  preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
  return preds
  
preds = paraphrase("paraphrase: What is the best framework for dealing with a huge text dataset?")

for pred in preds:
  print(pred)

In [ ]:
!pip install sentencepiece
!pip install torch==1.4.0
!pip install transformers==2.9.0
!pip install pytorch_lightning==0.7.5

     |████████████████████████████████| 1.2 MB 5.3 MB/s 
     |████████████████████████████████| 753.4 MB 6.8 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+cu111
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.1+cu111 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
torchtext 0.11.0 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
torchaudio 0.10.0+cu111 requires torch==1.10.0, but you have torch 1.4.0 which is incompatible.
     |████████████████████████████████| 635 kB 5.3 MB/s 
     |████████████████████████████████| 5.6 MB 43.1 MB/s 
     |████████████████████████████████| 895 kB 34.0 MB/s 
     |████████████████████████████████| 233 kB 5.2 MB/s 
     |██████████████████████

In [ ]:
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer


def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_paraphraser')
tokenizer = T5Tokenizer.from_pretrained('t5-base')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)


Downloading:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

device  cuda


In [ ]:
data.Questions[0]

'What are the SBIR and STTR Programs? What are the differences between the two programs, and which is program is more appropriate for my project?'

In [ ]:
# sentence = "Which course should I take to get started in data science?"
# sentence = "What are the ingredients required to bake a perfect cake?"
# sentence = "What is the best possible approach to learn aeronautical engineering?"
# sentence = "Do apples taste better than oranges in general?"
sentence = data.Questions[0]


text =  "paraphrase: " + sentence + " </s>"


max_len = 256

encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)


# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
beam_outputs = model.generate(
    input_ids=input_ids, attention_mask=attention_masks,
    do_sample=True,
    max_length=256,
    top_k=120,
    top_p=0.98,
    early_stopping=True,
    num_return_sequences=10
)


print ("\nOriginal Question ::")
print (sentence)
print ("\n")
print ("Paraphrased Questions :: ")
final_outputs =[]
for beam_output in beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    if sent.lower() != sentence.lower() and sent not in final_outputs:
        final_outputs.append(sent)

for i, final_output in enumerate(final_outputs):
    print("{}: {}".format(i, final_output))



Original Question ::
What are the SBIR and STTR Programs? What are the differences between the two programs, and which is program is more appropriate for my project?


Paraphrased Questions :: 
0: What are the differences between the SBIR and STTR programs?
1: What are the difference between SBIR and STTR program?
2: What is the difference between sbr and star?
3: What are the differences between the SBIR and the STTR programs?
4: What is the difference between SBIR and STTR?
5: What is difference between SBIR and STTR?
6: What are the differences between SBIR and STTR?
7: Which is the best program for a high quality project?
8: Which are the SBIR and the STTR programs?
9: What is the difference between SBIR and STTR? What are the similarities between the two programs?


In [ ]:
def paraphrased(sentence,num_paraphrased=5):
  text =  "paraphrase: " + sentence + " </s>"
  max_len = 256
  encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
  input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)
  # set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
  beam_outputs = model.generate(
      input_ids=input_ids, attention_mask=attention_masks,
      do_sample=True,
      max_length=256,
      top_k=120,
      top_p=0.98,
      early_stopping=True,
      num_return_sequences=num_paraphrased
  )


  # print ("\nOriginal Question ::")
  # print (sentence)
  # print ("\n")
  # print ("Paraphrased Questions :: ")
  final_outputs =[]
  for beam_output in beam_outputs:
      sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
      if sent.lower() != sentence.lower() and sent not in final_outputs:
          final_outputs.append(sent)

  # for i, final_output in enumerate(final_outputs):
  #     print("{}: {}".format(i, final_output))
  return final_outputs

In [ ]:
def add_values_in_dict(sample_dict, key, list_of_values):
    ''' Append multiple values to a key in 
        the given dictionary '''
    if key not in sample_dict:
        sample_dict[key] = list()
    sample_dict[key].extend(list_of_values)
    return sample_dict

In [ ]:
x=paraphrased(data.Questions[0],5)
x

['What are the difference between SBIR and STTR?',
 'What are the differences between SBIR and STTR program?',
 'What is the difference between SBIR and STTR programs and how might they be used for different projects?',
 'What are the differences between SBIR and STTR programs?',
 'What are the difference between SBIR and STTR programs?']

In [ ]:
para_dict={}
# para_dict = add_values_in_dict(para_dict,data.Questions[0],x)

In [ ]:
for i in data.Questions:
  x = paraphrased(i,5)
  para_dict = add_values_in_dict(para_dict,i,x)



In [ ]:
para_dict

{}

In [ ]:
len(para_dict.keys())

81

In [ ]:
para_dict

{'Are examples available of recently-funded NSF Phase I proposals and/or awards?': ['Are examples of recent NSF funded Phase I grants?',
  'Are there any recent funding at the NSF Phase I or III level?',
  'Are examples of recently funded NSF Phase I proposals and/or awards available?',
  'What is the best example of an NSF grant in phase I for a new individual?',
  'Are current NSF Phase I grants good examples?'],
 'Are first-time entrepreneurs appropriate candidates to participate in the program?': ['Are first time entrepreneurs good candidates for the entrepreneurship course?',
  'What are the benefits of the entrepreneurship program for beginner entrepreneurs?',
  'Are there any candidate for Entrepreneurship Program (Eiter)?',
  'Are all entrepreneurs considered candidates for the PMC/ECE programs?',
  'Do entrepreneurs want to participate in the Program?'],
 'Are new startup companies appropriate candidates for the program?': ['Are all new startups suitable candidates for the MBA

In [ ]:
import json
with open("T5_paraphrased.json","w") as f:
  json.dump(para_dict,f)

SyntaxError: ignored

In [ ]:
count=0
for i in para_dict.values():
  count+=len(i)
print(count)

352


In [ ]:
pd.DataFrame(para_dict.items(),columns=["Questions","Paraphrased"])

,Questions,Paraphrased
0,What are the SBIR and STTR Programs? What are ...,[What are the differences between the SBIR and...
1,What are the funding priorities for NSF SBIR/S...,[What are the funding priorities for NSF SBIR ...
2,What is innovative research and how do I gauge...,[What are some examples of innovative research...
3,How does NSF select which projects to support?...,[What are the criteria for NSF selecting new p...
4,What activities and expenses are appropriate t...,[What activities and expenses are appropriate ...
...,...,...
76,How and when does NSF release Phase I funding ...,[How do we get Phase I funding for a business?...
77,What other support do Phase I awardees receive?,[What is the support of the Phase I grant reci...
78,When and how do I apply for a Phase II award?,"[How do I apply for a Phase 2 award?, How do I..."
79,Does NSF have a Phase III SBIR/STTR program?,"[Does NSF have SBIR/STTR funding program?, Doe..."


In [ ]:
para_dict.keys()

dict_keys(['What are the SBIR and STTR Programs? What are the differences between the two programs, and which is program is more appropriate for my project?', 'What are the funding priorities for NSF SBIR/STTR?', 'What is innovative research and how do I gauge whether my research is a good fit?', 'How does NSF select which projects to support? Who manages the review and selection process?', 'What activities and expenses are appropriate to be funded in a Phase I project? What activities and expenses are not permitted?', 'Will NSF fund work on a product that has already been developed? Will the program fund small businesses to pay sales and marketing, business development, or other related costs?', 'Must Phase I proposers submit preliminary data as part of the proposal?', 'Are new startup companies appropriate candidates for the program?', 'Are first-time entrepreneurs appropriate candidates to participate in the program?', 'What are the expected outcomes (deliverables) of a Phase I proj

In [ ]:
print(json.dumps(para_dict,indent=10))

{
          "What are the SBIR and STTR Programs? What are the differences between the two programs, and which is program is more appropriate for my project?": [
                    "What are the differences between the SBIR and STTR programs?",
                    "What are the difference between a SBIR and STTR Program?",
                    "What is the difference between SBIR and STTR programs?",
                    "What is the difference between an SBIR and a STTR program?",
                    "What are the differences between the SBIR and STTR Programs?"
          ],
          "What are the funding priorities for NSF SBIR/STTR?": [
                    "What are the funding priorities for NSF SBIR STTR?",
                    "What funding priorities do we have for SBIR/STTR?",
                    "What are the funding priorities for NSF SBIR-STTR?",
                    "What are the funding priorities for NSF SBIR?",
                    "What are the funding priorities for the N